In [64]:
import re

In [233]:
def replace_ieee_types(to_sv: str):
    global elementary_types
    if(re.search(elementary_types, to_sv)):
        sv_eq = re.sub(elementary_types,'logic',to_sv)
        return sv_eq
    else:
        return to_sv

def replace_downto(to_sv: str):
    ret_str = to_sv
    global range_types
    if(re.search(range_types, ret_str)):
        ret_str = ret_str.replace('(','[')
        ret_str = ret_str.replace(')',']')
        if(re.search('[\.\w]*\'range(?=.*\])',ret_str)):
            operand = re.search('(?<=\[)[\.\w\d]*',ret_str).group(0)
            ret_str = re.sub('(?<=\[)[\.\w\d]*\'range(?=])','$left('+operand+') : $right('+operand+')',ret_str)
        if(re.search('\w*\'high(?=.*\])',ret_str)):
            operand = re.search('(?<=\[)[\w\d]*(?=\'high)',ret_str).group(0)
            ret_str = re.sub('(?<=\[)[\w\d]*\'high','$high('+operand+')',ret_str)
        if(re.search('\w*\'low(?=.*\])',ret_str)):
            operand = re.search('[\w\d]*(?=\'low\])',ret_str).group(0)
            ret_str = re.sub('[\w\d]*\'low(?=.*\])','$low('+operand+')',ret_str)
        if(ret_str.find(' downto ') > -1 or ret_str.find(' to ') > -1):
            ret_str = re.sub(' downto | to ', ' : ',ret_str)
    return ret_str

def clean_vhdl_residuals(to_sv:str):
    ret_str = to_sv
    if(ret_str.find('is array') > -1):
        ret_str = re.sub(' is array ',' ',ret_str)
        ret_str = re.sub(' of ',' ',ret_str)
        return ret_str.replace('type','typedef',1)
    elif(re.search('is\s*\(',ret_str)):
        ret_str = ret_str.replace('type', 'typedef enum',1).replace('(','{').replace(')','}')
        idx_is = ret_str.find('is')
        idx_curly = ret_str.find('}')
        name = ret_str[0:14]+ret_str[idx_is+2:idx_curly+1]+' '+ret_str[15:idx_is]
        return name
    if(re.search('(integer|natural|positive)\s*range', ret_str)):
        idx_walrus = ret_str.find(':=')
        name = re.search('\w*(?=\s*$)',ret_str).group(0)
        if(idx_walrus == -1):
            return 'integer '+name
        else:
            return 'integer '+name+ret_str[idx_walrus:]
    if(re.search('\[others => \'\d\'',ret_str)):
        if(ret_str.find('\'1\'') > 0):
            ret_str = re.sub('\[others => .*\]', '\'1',ret_str)
        elif(ret_str.find('\'0\'') > 0):
            ret_str = re.sub('\[others => .*\]', '\'0',ret_str)
        return ret_str
    #elif(re.search('integer'))
    else:
        return ret_str

def generics_to_macros(to_sv:str, macros:list):

    ret_str = to_sv
    for macro in macros:
        if(ret_str.find(macro) >= 0):
            ret_str = re.sub('\\b'+macro+'\\b', '`'+macro, ret_str)
            #print(ret_str)
            return ret_str
    return ret_str

def vhdl_bit_vec_to_sv(to_sv:str):
    ret_str = to_sv
    idx_eq = ret_str.find('=')
    idx_quote = ret_str.find('"')
    
    if(idx_eq > -1 and idx_quote > -1):
        idx_quote_end = ret_str[idx_quote+1:].find('"')
        replacement = str(idx_quote_end)+'\'b'+ret_str[idx_quote+1:idx_quote+idx_quote_end+1]
        ret_str = re.sub('\"\d*\"',replacement,ret_str)

    return ret_str
        

    

In [240]:
#f = open('/home/jonathanjonsson/MasterThesis/grlib-gpl-2022.1-b4273/lib/gaisler/noelv/core/noelv.vhd','r')
f = open('/home/jonathanjonsson/MasterThesis/grlib-gpl-2022.1-b4273/lib/gaisler/noelv/pkg/noelv.vhd','r')
#f = open('/home/jonathanjonsson/MasterThesis/grlib-gpl-2022.1-b4273/lib/grlib/riscv/riscv.vhd','r')
code = f.read().splitlines()
f.close()

f = open('/home/jonathanjonsson/MasterThesis/iunv_macros.md','r')
macros = f.read().splitlines()
macros = [macro.split(' ')[1] for macro in macros] #could be dangerous to use element 1 directly
f.close()


type_re = re.compile('^\s*type.*')
subtype_re = re.compile("^\s*subtype.*")
constant_re = re.compile("^\s*constant.*(integer|std_logic).*")
comment_re = re.compile("^\s*--")
elementary_types = re.compile('std_logic(_vector)?|std_ulogic(_vector)?')#integer')
range_types = re.compile('\'range|downto|to|\'high|\'low')

constants_vhdl = []
constants_sv = []
types_vhdl = []
types_sv = []
subtypes_vhdl = []
subtypes_sv = []


for ln,line in enumerate(code):
    type_match = re.search(type_re,line)
    subtype_match = re.search(subtype_re,line)
    constant_match = re.search(constant_re,line)
    count = 0
    if(type_match):
        matched_line = type_match.group(0)
        if(matched_line.find('is record') >= 0):
            count = ln
            str_append = ''
            while(code[count].find('end record') is -1):
                r_line = code[count]
                if(count is ln):
                    title = r_line.replace(' is record','').replace(' type','}')
                else:
                    if(re.search(comment_re, r_line)):
                        count += 1
                        continue;
                    idx_col = r_line.find(':')
                    idx_com = r_line.find('--')
                    idx_sem = r_line.find(';')
                    str_append += ('\t'+r_line[idx_col+2:idx_sem]+r_line[0:idx_col]+r_line[idx_com:]+'\n').replace('--',';//')
                count += 1
            types_vhdl.append("typedef struct {\n"+str_append+title.replace('--','//'))
        else:
            types_vhdl.append(type_match.group(0)+';')

    if(subtype_match):
        subtypes_vhdl.append(subtype_match.group(0))
    if(constant_match):      
        constants_vhdl.append(constant_match.group(0))


#replace vhdl with systemverilog equivalents
for subtype in subtypes_vhdl:
    pivot_is = subtype.find('is')
    pivot_semi = subtype.find(';')
    sub_line = (subtype[pivot_is+2:pivot_semi]+subtype[0:pivot_is]).replace('subtype','')
    sub_line = replace_ieee_types(sub_line)
    sub_line = replace_downto(sub_line)
    sub_line = clean_vhdl_residuals(sub_line)
    #sub_line = generics_to_macros(sub_line, macros)
    subtypes_sv.append('typedef '+sub_line+';\n')

for typ in types_vhdl:
    sub_line = typ
    if(typ.find('is array') > 0): #if we have an array, convert it to systemverilog counterpart
        sub_line = typ
        pivot_is_array = sub_line.find('is array')
        sub_line = sub_line.replace(' type','')
        pivot_sem = sub_line.find(';')
        pivot_of = sub_line.find('of')
        sub_line = 'typedef '+sub_line[pivot_of+3:pivot_sem]+" "+sub_line[0:pivot_is_array-5]+sub_line[pivot_is_array+3:pivot_of]
    
    sub_line = replace_ieee_types(sub_line)
    sub_line = replace_downto(sub_line)
    sub_line = clean_vhdl_residuals(sub_line)
    #sub_line = generics_to_macros(sub_line, macros)
    types_sv.append(sub_line+';\n')
    # .replace('type','',1).replace('--','//')

for const in constants_vhdl:
    sub_line = const
    sub_line = sub_line.replace('constant','').replace('--','//')
    idx_walrus = sub_line.find(':=')
    idx_col = sub_line.find(':')
    idx_sem = sub_line.find(';')
    idx_com = sub_line.find('--')
    sub_line = sub_line[idx_col+1:idx_walrus]+sub_line[0:idx_col]+sub_line[idx_walrus:]
    sub_line = replace_ieee_types(sub_line)
    sub_line = replace_downto(sub_line)
    sub_line = clean_vhdl_residuals(sub_line)
    sub_line = generics_to_macros(sub_line, macros).replace(':=', '=')
    sub_line = vhdl_bit_vec_to_sv(sub_line)
    constants_sv.append("const "+sub_line+'\n')


f = open('iunv_sv_package.sv','w')
f.write('package iunv_sv_package;\n')
f.writelines(constants_sv)
f.writelines(subtypes_sv)
f.writelines(types_sv)
f.write('endpackage')
f.close()

FileNotFoundError: [Errno 2] No such file or directory: '/home/jonathanjonsson/MasterThesis/grlib-gpl-2022.1-b4273/lib/gaisler/noelv/core/noelv.vhd'

In [208]:
tmp = constants_vhdl.copy()
#tmp.append(' std_logic_vector(lanes_type\'range)   hit_vec_type ')
for ln in range(len(tmp)):
    sub_line = tmp[ln]
    sub_line = sub_line.replace('constant','').replace('--','//')
    idx_walrus = sub_line.find(':=')
    idx_col = sub_line.find(':')
    idx_sem = sub_line.find(';')
    idx_com = sub_line.find('--')
    sub_line = sub_line[idx_col+1:idx_walrus]+sub_line[0:idx_col]+sub_line[idx_walrus:]
    sub_line = replace_ieee_types(sub_line)
    sub_line = replace_downto(sub_line)
    sub_line = clean_vhdl_residuals(sub_line)
    sub_line = generics_to_macros(sub_line, macros).replace(':=', '=')
    print('const '+sub_line)


const  integer    memory_lane = 0;
const  integer    branch_lane = 1 - `single_issue;
const  integer    one         = 1 - `single_issue;
const  logic[1 : 0]    rv = u2vec[log2[XLEN / 32] + 1, 2];
const  logic    va         = gaisler.mmucacheconfig.va(`riscv_mmu);
const  logic    pa         = gaisler.mmucacheconfig.pa(`riscv_mmu);
const  integer             pmp_msb    = `physaddr - 1;    // High bit for PMP checks
const  integer    ext_f        = to_floating[`fpulen, 32];
const  integer    ext_d        = to_floating[`fpulen, 64];
const  integer    ext_q        = to_floating[`fpulen, 128];
const  integer    ext_n        = 0;
const  integer    RFBITS       = 5;
const  integer    FUSELBITS    = 10;
const  logic[0 : 1 - `single_issue]    lanes        = '0;  // Used as range.
const  integer    CAUSEBITS    = 4 + `ext_h;
const  integer    DWAYMSB      = log2x(`dways) - 1;
const  integer    IWAYMSB      = log2x(`iways) - 1;
const  integer    TBUFBITS     = 4 + log2(`tbuf);
const  logic[25 : 0]

In [113]:
macros

['hindex',
 'fabtech',
 'memtech',
 'physaddr',
 'pcbits',
 'rstaddr',
 'disas',
 'perf_cnts',
 'perf_evts',
 'illegalTval0',
 'no_muladd',
 'single_issue',
 'iways',
 'dways',
 'mmuen',
 'riscv_mmu',
 'pmp_no_tor',
 'pmp_entries',
 'pmp_g',
 'asidlen',
 'vmidlen',
 'ext_m',
 'ext_a',
 'ext_c',
 'ext_h',
 'mode_s',
 'mode_u',
 'dmen',
 'fpulen',
 'trigger',
 'late_branch',
 'late_alu',
 'pbaddr',
 'tbuf',
 'scantest',
 'rfreadhold',
 'fpu_lane',
 'csr_lane',
 'endian']